<a href="https://colab.research.google.com/github/opedoussaut/orbit/blob/release%2Fv0.2.0/ORBIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORBIT
## From ontology to usable reference structures (semantic contracts)

**Version:** v0.2.0 (in progress)

ORBIT turns an ORIONT-aligned representation into a **semantic contract framework**:
- **OWL/RDF** describes meaning and structure (open world)
- **SHACL** expresses *contracts* for structural completeness, consistency, and comparability (closed-world checks)

This notebook is a runnable walkthrough of the v0.2.0 contract packs.


## ORBIT v0.2.0 (Contract-based validation)

This version introduces semantic contracts for:
- LCA structural integrity
- LCIA metrics comparability
- Circularity assumptions
- Reporting and EPD readiness

Contracts are expressed as SHACL shapes and applied to RDF graphs
instantiated from LCA study data.

## How this notebook is structured (v0.2.0 progression)

v0.2.0 is organized as a progression of contracts:

### 1) LCA-CORE (Study integrity)
Ensures the study has an explicit **goal/scope envelope**:
- Functional Unit
- Reference Flow
- System Boundary
- Activities with lifecycle stage + temporal validity

### 2) EF3.1 Metrics vocabulary (controlled metric identity)
Introduces a controlled list of LCIA metrics (EF3.1 midpoint) as **SKOS concepts**.

### 3) LCIA-METRICS (Comparability readiness)
Ensures each LCIA result declares:
- metric identity (from EF3.1 scheme)
- method binding (IRI)
- unit binding (IRI)
- numeric value
- scope target (product/stage/activity)

Each contract is demonstrated with:

✅ a conforming example  
❌ a non-conforming example  
and a SHACL conformance report.

In [ ]:
!pip -q install rdflib pyshacl

from rdflib import Graph
from pyshacl import validate
import urllib.request

In [ ]:
BRANCH = "release/v0.2.0"
BASE_RAW = f"https://raw.githubusercontent.com/opedoussaut/orbit/{BRANCH}"

CONTRACT_PATHS = {
    "lca_core": "contracts/lca-core.shacl.ttl",
    "lcia_metrics": "contracts/lcia-metrics.shacl.ttl",
}

VOCAB_PATHS = {
    "metrics": "vocab/metrics.skos.ttl",
}

EXAMPLE_PATHS = {
    "lca_ok":  "examples/lca-core_conforming.ttl",
    "lca_bad": "examples/lca-core_nonconforming.ttl",
    "lcia_ok":  "examples/lcia-metrics_conforming.ttl",
    "lcia_bad": "examples/lcia-metrics_nonconforming.ttl",
}

In [ ]:
def assert_url_ok(url: str):
    try:
        with urllib.request.urlopen(url) as _:
            return True
    except Exception as e:
        raise RuntimeError(f"Cannot load URL:\n{url}\n\n{e}")

def load_ttl_from_url(url: str) -> Graph:
    assert_url_ok(url)
    g = Graph()
    data = urllib.request.urlopen(url).read().decode("utf-8")
    g.parse(data=data, format="turtle")
    return g

def run_contract(data_graph: Graph, shacl_graph: Graph, label: str):
    conforms, report_graph, report_text = validate(
        data_graph=data_graph,
        shacl_graph=shacl_graph,
        inference="rdfs",
        abort_on_first=False
    )
    print(f"\n=== {label} ===")
    print("CONFORMS:", conforms)
    print(report_text)
    return conforms, report_text

## 1) LCA-CORE — Study integrity contract

This first contract validates that a study is structurally interpretable:
it must declare a functional unit, a reference flow, a system boundary, and at least one activity.
It also checks minimal temporal consistency for activities.

Goal: **fail fast** on incomplete or internally inconsistent studies before discussing metrics or reporting.

In [ ]:
lca_shacl = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['lca_core']}")
lca_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_ok']}")
lca_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_bad']}")

run_contract(lca_ok,  lca_shacl, "LCA-CORE / Conforming example")
run_contract(lca_bad, lca_shacl, "LCA-CORE / Non-conforming example")

## 2) EF3.1 midpoint metrics vocabulary (controlled identity)

Before validating LCIA results, ORBIT introduces a controlled vocabulary of metrics.
This prevents “metric-by-string” ambiguity (e.g., “carbon footprint”, “climate change”, “GWP”)
and enables contracts to require that metric identity comes from the EF3.1 scheme.

In [ ]:
metrics_vocab = load_ttl_from_url(f"{BASE_RAW}/{VOCAB_PATHS['metrics']}")
print("Triples in metrics vocabulary:", len(metrics_vocab))

## 3) LCIA-METRICS: Comparability readiness contract (EF3.1 aligned)

This contract does **not** compute LCIA.
It validates whether reported LCIA results are structurally comparable:

- metric identity is explicit and belongs to the EF3.1 scheme
- method is explicitly referenced (IRI)
- unit is explicitly referenced (IRI)
- numeric value exists
- scope target (appliesTo) is explicit and typed (product/stage/activity)

This makes “results” defensible artifacts rather than unqualified numbers.

In [ ]:
lcia_shacl = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['lcia_metrics']}")
lcia_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lcia_ok']}")
lcia_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lcia_bad']}")

lcia_ok_all  = lcia_ok + metrics_vocab
lcia_bad_all = lcia_bad + metrics_vocab

run_contract(lcia_ok_all,  lcia_shacl, "LCIA-METRICS / EF3.1 / Conforming example")
run_contract(lcia_bad_all, lcia_shacl, "LCIA-METRICS / EF3.1 / Non-conforming example")

## What v0.2.0 establishes

With LCA-CORE + EF3.1 metric identity + LCIA-METRICS, ORBIT v0.2.0 demonstrates a shift:

- from “we have results”  
- to “we have results that are structurally interpretable and comparable-ready”

Next contract packs (circularity, reporting/EPD readiness) build on this foundation.